This notebook can be run on colab: [here](https://colab.research.google.com/github/DataScience4Geoscience/Toulouse2020/blob/master/Notebooks/Classification/N1_Classification_LDA_QDA.ipynb)

# Classification 
In this notebook, the classification with LDA and QDA is illustrated on the Digits data set.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_digits
# Load digits
X, y = load_digits(return_X_y=True)
print(X.shape)
print(X.max())

The data set is made available by NIST to extract normalized bitmaps of handwritten digits from a preprinted form. From a total of 43 people, 30 contributed to the training set and different 13 to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of 4x4 and the number of on pixels are counted in each block. This generates an input matrix of 8x8 where each element is an integer in the range 0..16 (https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits)

We can visualize the data by reshaping correctly each sample:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plot the first element
plt.figure()
plt.imshow(X[0,:].reshape(8,8),cmap="gray")
plt.title("Label = {}".format(y[0]))

# Plot the hundredth element
plt.figure()
plt.imshow(X[100,:].reshape(8,8),cmap="gray")
plt.title("Label = {}".format(y[100]))

To perform the classification and the validation of the model, we should split our data sets into two disjoint parts. The classification accuracy is assessed using the global accuracy.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA,\
                                        QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as OA

# Split the data -> train_size=0.20 means we keep 20% of the data for training and 80% for validation
# The stratification ensures that the proportion of each class from the orginal data is preserved in the train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=0, stratify=y)

MODELS = [LDA(), QDA()]
oa = []
for model in MODELS:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    oa.append(OA(y_test, y_pred))
print(oa)

# Setting up hyperparameters
Both LDA and QDA have hyperparameters, which are related to the estimation of the covariance matrices. They usually have significant influences on the final classification results. 

For instance, QDA implemented in Sklearn has a regulation parameter that control the condition number of the class covariance matrix: $$(1-\lambda)\Sigma + \lambda\mathbf{I}.$$ The following code shows the influence of this hyperparameter on the overall accuracy.

In [ ]:
import scipy as sp
reg_param = sp.linspace(0,1)
print(reg_param)

In [ ]:
oa = []
for reg_param_ in reg_param:
    model = QDA(reg_param=reg_param_)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    oa.append(OA(y_test, y_pred))
plt.plot(reg_param, oa)
plt.grid()

In [ ]:
print("Best hyperparameters value: {}".format(reg_param[sp.argmax(oa)]))
print("Best classification value: {}".format(oa[sp.argmax(oa)]))
